## Importing modules

In [2]:
import matplotlib.pyplot as plt
import nilearn.image as image
from nilearn.image import resample_to_img
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_img, plot_anat, plot_stat_map, plot_roi, plot_design_matrix, plot_contrast_matrix, view_img
import numpy as np
import os
import glob
import nibabel as nib
import pandas as pd
from pathlib import Path
import tracemalloc

## Defining global variables

In [33]:
#path_data = '/data/rainville/PAINxEFFORT/PREPROCESSED_SPM'
path_data= 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images'
path_events = 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/Output/Python'
path_mask = 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images/output brain mask/gm_group_binarized_mask.nii'
path_output = 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/derivatives'

sub = "sub-001"

t_r = 0.832

### Defining the contrasts

In [34]:
def make_localizer_contrasts(design_matrix):
    """Return a dictionary of four contrasts, given the design matrix."""
    # first generate canonical contrasts
       
    contrast_matrix_run1 = np.eye(design_matrix.shape[1])
    
    contrasts = {
        column: contrast_matrix_run1[i]
        for i, column in enumerate(design_matrix.columns)
    }

    contrasts["pain"] = (
        contrasts["Pain_30"]
        + contrasts["Pain_5"]
    )

    # one contrast adding all conditions involving warm
    contrasts["warm"] = (
        contrasts["Warm_30"]
        + contrasts["Warm_5"]
    )
      # one contrast adding all conditions involving thermal stimulation
    contrasts["thermal_stimulation"] = (
        contrasts["warm"]
        + contrasts["pain"]
    )
      # one contrast adding all conditions involving all contractions during thermal stimulations
    contrasts["all_contractions_during_stim"] = (
        contrasts["ContractionWarm_5"] + contrasts["ContractionPain_5"] + contrasts["ContractionWarm_30"] 
        + contrasts["ContractionPain_30"]
    )
      # one contrast adding all conditions involving all contractions solo
    contrasts["all_contractions_solo"] = (
        contrasts["ContractionSolo_5"] + contrasts["ContractionSolo_30"] 
    )


    # one contrast adding all conditions involving contraction at 5%
    contrasts["contraction_5"] = (
        contrasts["ContractionSolo_5"] + contrasts["ContractionWarm_5"] + contrasts["ContractionPain_5"]
    )

    # one contrast adding all conditions involving contraction at 30%
    contrasts["contraction_30"] = (
        contrasts["ContractionSolo_30"] + contrasts["ContractionWarm_30"] + contrasts["ContractionPain_30"]
    )

       # one contrast adding all conditions involving contraction during warm
    contrasts["contraction_warm"] = (
        contrasts["ContractionWarm_30"] + contrasts["ContractionWarm_5"]
    )
    
    
          # one contrast adding all conditions involving contraction during pain
    contrasts["contraction_pain"] = (
        contrasts["ContractionPain_30"] + contrasts["ContractionPain_5"]
    )
    

    # Short dictionary of more relevant contrasts
    contrasts = {
        "pain - warm": (
            contrasts["pain"]
            - contrasts["warm"]
        ),
        "contraction 30%-5%": contrasts["contraction_30"] - contrasts["contraction_5"],
        "contraction Warm-Pain": (
            contrasts["contraction_pain"] - contrasts["contraction_warm"]
        ),
        "contraction 5%_pain - warm": (
            contrasts["ContractionPain_5"]
            - contrasts["ContractionWarm_5"]
        ),
        "contraction 30%_pain - warm": (
            contrasts["ContractionPain_30"]
            - contrasts["ContractionWarm_30"]
        ),
         "contraction pain_30% - 5%": (
            contrasts["ContractionPain_30"]
            - contrasts["ContractionPain_5"]
        ),
         "contractionSolo 30%-5%": (
            contrasts["ContractionSolo_30"]
            - contrasts["ContractionSolo_5"]
        ),
        "all contraction thermal stim - thermal stim": (
            contrasts["all_contractions_during_stim"]
            - contrasts["thermal_stimulation"]
        ),
    }
    return contrasts

# run first level model for all run of a participant 

In [40]:
#all runs
def run_first_level_trial(path_data, path_events, path_mask, path_output, sub, t_r=0.832, contrasts=['pain - warm', 'contraction 30%-5%', 'contractionSolo 30%-5%', 'all contraction thermal stim - thermal stim']):
    """
    Parameters
    ----------
    path_data: str
        Directory containing the bold files
    path_events: str
        Directory containing the event files
    path_mask: str
        Path to the group mask
    path_output: str
        Outputs path
    sub: str
        Subject id
    t_r: float
        Repetition time
    """
    # Create output path if doesn't exit
    Path(path_output).mkdir(parents=True, exist_ok=True)
    
    #output_dir = os.path.join(path_output)
    #os.makedirs(output_dir, exist_ok=True)
    #print("Output directory:", output_dir)  # Debug
    
    # Load mask
    mask = nib.load(path_mask)
    # Events files
    events = [os.path.join(path_events, sub, e) for e in os.listdir(os.path.join(path_events, sub)) if "events" in e]
    events.sort()
    print(events)
    events = [pd.read_excel(e) for e in events]
    
    
    # Bold path
    bolds = [os.path.join(path_data,sub,"func",b) for b in os.listdir(os.path.join(path_data, sub ,"func")) if "swvrsub" in b and not b.startswith("._")]
    bolds.sort()
    print(bolds)

    dict_trials = {}


    for idx, (event, bold) in enumerate(zip(events, bolds), start=1):
        p = Path(bold)
        filename = "_".join(p.stem.split("_")[:-1] )
        print(filename)
        
        # Define model
        first_level_model = FirstLevelModel(t_r, mask_img=mask)
        # Fit the model
        fmri_glm = first_level_model.fit(bold, events=event)
    
        #  list of design matrices for each run
        design_matrices = fmri_glm.design_matrices_
        design_matrices[0].to_csv(os.path.join(path_output, sub, filename+"_desc-design_matrices.tsv.gz"))
        
            
        # Call the function to generate contrasts for all runs
        localizer_contrasts = []
        for design_matrix in fmri_glm.design_matrices_:
            localizer_contrasts.append(make_localizer_contrasts(design_matrix))
    
        # Compute the contrasts
        dict_maps = {}
        for contrast in contrasts:
            localizer_contrast = [i[contrast] for i in localizer_contrasts]
            beta_map = fmri_glm.compute_contrast(localizer_contrast, output_type='effect_size')
            nib.save(beta_map, os.path.join(path_output, sub, filename+f'_desc-{contrast}-maps.nii.gz'))
            dict_maps[contrast] = beta_map
        dict_trials[f"run-0{idx}"] = dict_maps
       
    
    return dict_trials
    
    

In [41]:
bolds = [os.path.join(path_data,sub,"func",b) for b in os.listdir(os.path.join(path_data, sub ,"func")) if "swvrsub" in b and not b.startswith("._")]

In [42]:
dict_sub001 = run_first_level_trial(path_data, path_events, path_mask, path_output, sub, t_r=0.832, contrasts=['pain - warm', 'contraction 30%-5%', 'contractionSolo 30%-5%', 'all contraction thermal stim - thermal stim'])

['C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/Output/Python\\sub-001\\sub-001_events_run1.xlsx', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/Output/Python\\sub-001\\sub-001_events_run2.xlsx', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/Output/Python\\sub-001\\sub-001_events_run3.xlsx', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/Output/Python\\sub-001\\sub-001_events_run4.xlsx']
['C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images\\sub-001\\func\\swvrsub-001_task-pain_run-01_bold.nii', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images\\sub-001\\func\\swvrsub-001_task-pain_run-02_bold.nii', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images\\sub-001\\func\\swvrsub-001_task-pain_run-03_bold.nii', 'C:/Users/ilari/OneDrive - Universite de Montreal/EXP/MRI effort x pain/MRI images\\sub-0

In [43]:
dict_sub001

{'run-01': {'pain - warm': <nibabel.nifti1.Nifti1Image at 0x1a53d6853d0>,
  'contraction 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a53fe5c160>,
  'contractionSolo 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a5403439d0>,
  'all contraction thermal stim - thermal stim': <nibabel.nifti1.Nifti1Image at 0x1a53d3ddbe0>},
 'run-02': {'pain - warm': <nibabel.nifti1.Nifti1Image at 0x1a5403ebcd0>,
  'contraction 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a5403eb3d0>,
  'contractionSolo 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a5403ebbe0>,
  'all contraction thermal stim - thermal stim': <nibabel.nifti1.Nifti1Image at 0x1a5403eb550>},
 'run-03': {'pain - warm': <nibabel.nifti1.Nifti1Image at 0x1a5403eb370>,
  'contraction 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a5403ebb20>,
  'contractionSolo 30%-5%': <nibabel.nifti1.Nifti1Image at 0x1a53d685550>,
  'all contraction thermal stim - thermal stim': <nibabel.nifti1.Nifti1Image at 0x1a54034a0d0>},
 'run-04': {'pain - warm': <nibabel.nifti1.Nifti1